# Graph Algebra with `kglab`

## intro
`kglab` provides tools to access graph data from multiple source to build a `KnowledgeGraph` that can be easily used by data scientists. For a thorough explanation of how to use triples-stored data and how to load this data into `kglab` please see examples in the `examples/` directory. The examples in this directory (`examples/graph_algebra/`) will care to introduce graph algebra capabilities to be used on the graphs the user has loaded. 

## basic load and querying
In particular, once your data is loaded in a `KnowledgeGraph` with something like:

1. Instantiate a graph from a dataset:

In [50]:
# for use in tutorial and development; do not include this `sys.path` change in production:
import sys ; sys.path.insert(0, "../../")
import warnings
warnings.filterwarnings('ignore')

from os.path import dirname
import kglab
import os

namespaces = {
    "foaf": "http://xmlns.com/foaf/0.1/",
    "gorm": "http://example.org/sagas#",
    "rel":  "http://purl.org/vocab/relationship/",
    }

kg = kglab.KnowledgeGraph(
    name = "Happy Vikings KG example for SKOS/OWL inference",
    namespaces=namespaces,
    )

kg.load_rdf(dirname(dirname(os.getcwd())) + "/dat/gorm.ttl")


2. It is possible to create a subgraph by providing a SPARQL query, by defining a "subject" and "object":


In [51]:
query = """SELECT ?subject ?object
WHERE {
    ?subject rdf:type gorm:Viking .
    ?subject gorm:childOf ?object .
}
"""
df = kg.query_as_df(query)
df

,subject,object
0,gorm:Astrid,gorm:Leif
1,gorm:Astrid,gorm:Bodil
2,gorm:Leif,gorm:Bjorn
3,gorm:Bjorn,gorm:Gorm



## define a subgraph
In this case we are looking for the network of parent-child relations among members of Vikings family.

With this query we can define a __*subgraph* so to have access to *graph algebra* capabilities__: 

In [52]:
from kglab.subg import SubgraphMatrix

subgraph = SubgraphMatrix(kg=kg, sparql=query)


## compute Adjacency matrices
Let's compute the first basic adjacency matrix (usually noted with `A`):

In [53]:
adj_matrix = subgraph.to_adjacency()
adj_matrix

array([[0., 1., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.]])

what happened here is that all the subjects and objects have been turned into integer indices from 0 to number of nodes. So we can see that the entity with index 0 is adjancent (is connected, has a directed edge) to the entity with index 1. This is a directed graph because the relationship `gorm:childOf` goes from child to parent, let's turn this into an undirected graph so to see the relation in a more symmetric way (both the child-parent and parent-child).

We can check the labels attached to the matrix's indices with:

In [54]:
print("index", "->", "label")
for i in range(adj_matrix.shape[0]):
    print(
        i, "->", subgraph.inverse_transform(i)  # returns a label from an index
    )

index -> label
0 -> http://example.org/sagas#Astrid
1 -> http://example.org/sagas#Leif
2 -> http://example.org/sagas#Bodil
3 -> http://example.org/sagas#Bjorn
4 -> http://example.org/sagas#Gorm


We can see from the matrix, assigning labels to the indices, for examples that: Astrid is child of Leif and Bodil.

This is one of the great functionality provided by the semantic layer (data that is represented by W3C Linked Data standard), to represent relationships in both human-understandable and machine-readable way.

Another useful method is `describe()`, that returns some statistics of the graph if they can be computed:

In [55]:
subgraph.describe()

{'n_nodes': 5,
 'n_edges': 4,
 'center_msg': 'Found infinite path length because the digraph is not strongly connected',
 'diameter_msg': 'Found infinite path length because the digraph is not strongly connected',
 'eccentricity_msg': 'Found infinite path length because the digraph is not strongly connected',
 'center': None,
 'diameter': None,
 'eccentricity': None}


## other relevant matrices for a graph

To compute the *vertices degrees matrix* we need to port our directed graph (semantic data graph are always directed as by design triples are `subject->relation->object`) into an undirected ones. This obviously preserve the existence of the relationships but not their direction.

In [56]:
undirected_adj_mtx = subgraph.to_undirected()
undirected_adj_mtx

array([[0., 1., 1., 0., 0.],
       [1., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0.]])

We can see now the relationship is a generic symmetric "parenthood" relations, not just a child-parent directed relationship. We can still say that: Leif and Bodil and Astrid are first-degree kins (parent-child or siblings). 

Same easy way we can compute the vertices degrees matrix:

In [57]:
laplacian = subgraph.to_laplacian()
laplacian

array([[ 2, -1, -1,  0,  0],
       [-1,  2,  0, -1,  0],
       [-1,  0,  1,  0,  0],
       [ 0, -1,  0,  2, -1],
       [ 0,  0,  0, -1,  1]])

An incidence, or edge matrix `E`, uses by convention the rows to represent every node in the graph and the columns represent every edge. Some other convention does the opposite.

In [58]:
incidence = subgraph.to_incidence()
incidence

array([[1., 1., 0., 0.],
       [1., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 1.],
       [0., 0., 0., 1.]])

## try another query
Let's try the same tools with another query that will define a different subgraph from the main graph: 

In [59]:
query2 = """SELECT ?subject ?object
WHERE {
    ?subject rdf:type gorm:Viking .
    ?subject gorm:spouseOf ?object .
}
"""
df = kg.query_as_df(query2)

This time we try a *symmetric* relation `gorm:spouseOf`. Let's try to understand better how the RDF definition of our semantic layer works:

In [60]:
text = kg.save_rdf_text()
print(text)

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix gorm: <http://example.org/sagas#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

gorm:Astrid a gorm:Viking ;
    gorm:childOf gorm:Bodil,
        gorm:Leif ;
    foaf:topic_interest gorm:Fighting .

gorm:childOf rdfs:domain gorm:Viking ;
    rdfs:range gorm:Viking ;
    owl:inverseOf gorm:ancestorOf .

gorm:spouseOf a owl:SymmetricProperty ;
    rdfs:domain gorm:Viking ;
    rdfs:range gorm:Viking .

gorm:Berserkr a foaf:Thing ;
    skos:broader gorm:Fighting .

gorm:Bjorn a gorm:Viking ;
    gorm:childOf gorm:Gorm ;
    foaf:topic_interest gorm:Pilaging .

gorm:Bodil a gorm:Viking ;
    gorm:spouseOf gorm:Leif .

gorm:Gorm a gorm:Viking ;
    foaf:topic_interest gorm:Berserkr .

gorm:Pilaging a foaf:Thing ;
    skos:broader gorm:Fighting .

gorm:Leif a gorm:Viking ;
    gorm:childOf gorm:Bjorn .

gorm:Fighting a foaf:Th

As you can see `gorm:spouseOf` is defined as what the OWL standard calls a `owl:SymmetricProperty`, for this relation **domain** (the definition of the set of subject) and **range** (the definition of the set of object) are the same: so the triple looks like: `gorm:Viking`->`gorm:spouseOf`->`gorm:Viking`. Let's what data this relation returns.

In [61]:
subgraph2 = SubgraphMatrix(kg=kg, sparql=query2)
A = subgraph2.to_adjacency()
A

array([[0., 1.],
       [0., 0.]])

In [62]:
# Labels
for i in range(A.shape[0]):
    print(
        i, "->", subgraph2.inverse_transform(i)  # returns a label from an index
    )

0 -> http://example.org/sagas#Bodil
1 -> http://example.org/sagas#Leif


Turning this matrix representation into undirected, we can read more generic "is married to" relation:

In [63]:
undirected = subgraph2.to_undirected()
undirected

array([[0., 1.],
       [1., 0.]])